In [30]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [31]:
import json

In [32]:
with open('kaggle .json', 'r') as f:
    data = json.load(f)

In [33]:
data

{'username': 'aabithmusthaq', 'key': 'b91cc2cd1adafc85b6ddd87b77fea8fc'}

In [34]:
!pip install kaggle


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
import os
os.environ['KAGGLE_USERNAME'] = data['username']
os.environ['KAGGLE_KEY'] = data['key']

In [36]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [37]:
api.dataset_download_files('dineshpiyasamara/sentiment-analysis-dataset')

Dataset URL: https://www.kaggle.com/datasets/dineshpiyasamara/sentiment-analysis-dataset


In [38]:
import zipfile
with zipfile.ZipFile('sentiment-analysis-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [39]:
df = pd.read_csv(r"C:\Users\aabidh musthaq\Desktop\sentimenal project\review-sentimental-analysis\data\sentiment_analysis.csv")

In [40]:
df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


## Data preprocessing

In [41]:
df.shape

(7920, 3)

In [42]:
df.duplicated().sum()

0

In [43]:
df.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

### Text preprocessing

In [44]:
import re #regular expression
import string # to remove punctuation

In [45]:
df['tweet'].head()

0    #fingerprint #Pregnancy Test https://goo.gl/h1...
1    Finally a transparant silicon case ^^ Thanks t...
2    We love this! Would you go? #talk #makememorie...
3    I'm wired I know I'm George I was made that wa...
4    What amazing service! Apple won't even talk to...
Name: tweet, dtype: object

In [46]:
#CONVER UPPER CASE TO LOWER CASE
df['tweet'] = df['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split())) #lower case conversion split the words and join them

In [48]:
df['tweet'].head()

0    #fingerprint #pregnancy test https://goo.gl/h1...
1    finally a transparant silicon case ^^ thanks t...
2    we love this! would you go? #talk #makememorie...
3    i'm wired i know i'm george i was made that wa...
4    what amazing service! apple won't even talk to...
Name: tweet, dtype: object

In [50]:
#remove links 
df["tweet"] = df['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split())) #remove links 
#multilines means it will remove the links in the entire data

In [51]:
df['tweet'].head()

0    #fingerprint #pregnancy test  #android #apps #...
1    finally a transparant silicon case ^^ thanks t...
2    we love this! would you go? #talk #makememorie...
3    i'm wired i know i'm george i was made that wa...
4    what amazing service! apple won't even talk to...
Name: tweet, dtype: object

remove puncuation

In [52]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [53]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [54]:
df["tweet"] = df["tweet"].apply(remove_punctuations)

In [55]:
df['tweet'].head()

0    fingerprint pregnancy test  android apps beaut...
1    finally a transparant silicon case  thanks to ...
2    we love this would you go talk makememories un...
3    im wired i know im george i was made that way ...
4    what amazing service apple wont even talk to m...
Name: tweet, dtype: object

In [56]:
df["tweet"] = df['tweet'].str.replace('\d+', '', regex=True) #remove numbers

In [57]:
import nltk

In [58]:
nltk.download('stopwords', download_dir='../static/model') #download stopwords ex: is, am, are, the, etc

[nltk_data] Downloading package stopwords to ../static/model...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [59]:
with open('../static/model/corpora/stopwords/english', 'r') as file: #open the file
    sw = file.read().splitlines() #read the stopwords and split them

In [61]:
df["tweet"] = df["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw)) #remove stopwords

In [62]:
from nltk.stem import PorterStemmer #stemming is the process of reducing a word to its word stem
ps = PorterStemmer() #stemming

In [63]:
df["tweet"] = df["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split())) #stemming ex: running -> run

### Vocablary buliding 